# **Data Preparation**

In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GrainImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, mode='train'):

        self.dataframe = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.mode = mode

        self.class_to_index = {'wheat': 0, 'oats': 1, 'flax': 2, 'barley': 3}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        if self.mode == 'train':
            class_name = self.dataframe.iloc[idx, 1]
            label = self.class_to_index[class_name]
            label = torch.tensor(label, dtype=torch.long)
        else:
            label = torch.tensor(-1, dtype=torch.long)
        return image, label

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Model Architecture**

In [7]:
# Model definition
# class GrainClassifier(nn.Module):
#     def __init__(self):
#         super(GrainClassifier, self).__init__()

#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Input channels = 3 (RGB)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

#         self.fc1 = nn.Linear(256 * 8 * 8, 512)
#         self.fc2 = nn.Linear(512, 4)  # 4 classes: wheat, oats, flax, barley

#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.pool(F.relu(self.conv4(x)))

#         x = x.view(-1, 256 * 8 * 8)
#         x = self.dropout(x)

#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)

#         x = self.fc2(x)
#         return x

model = models.resnet50(pretrained=True)

# Freeze all the parameters in the convolutional layers
for param in model.parameters():
    param.requires_grad = False

# Modify the fully connected layer to output 4 classes
model.fc = nn.Linear(model.fc.in_features, 4)

# Move the model to the device
model = model.to(device)

# Define the transformation for the input images
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Specify the paths and directories for your dataset
train_csv_path = '/content/drive/MyDrive/NeuralNetworks/train/train.csv'
test_csv_path = '/content/drive/MyDrive/NeuralNetworks/test/test.csv'
train_img_dir = '/content/drive/MyDrive/NeuralNetworks/train/images'
test_img_dir = '/content/drive/MyDrive/NeuralNetworks/test/images'

# Create instances of your dataset
train_dataset = GrainImageDataset(csv_file=train_csv_path, img_dir=train_img_dir, transform=transform, mode='train')
test_dataset = GrainImageDataset(csv_file=test_csv_path, img_dir=test_img_dir, transform=transform, mode='test')

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# num_classes = 4
# model = GrainClassifier().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 159MB/s]


# **Optimizer**

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

# **Loss function**

In [9]:
loss_func = nn.CrossEntropyLoss()

# **Training**

In [10]:
def train(model, train_loader, loss_func, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

train(model, train_loader, loss_func, optimizer, num_epochs=100, device=device)

Epoch 1/100, Loss: 0.6975025957102304
Epoch 2/100, Loss: 0.4210741634552295
Epoch 3/100, Loss: 0.3666742819023656
Epoch 4/100, Loss: 0.35892940787496147
Epoch 5/100, Loss: 0.3433417002414609
Epoch 6/100, Loss: 0.2859839290216729
Epoch 7/100, Loss: 0.28679274272296457
Epoch 8/100, Loss: 0.26452695078902194
Epoch 9/100, Loss: 0.24068786261173394
Epoch 10/100, Loss: 0.2678277620656805
Epoch 11/100, Loss: 0.2637324848590971
Epoch 12/100, Loss: 0.25566910338270793
Epoch 13/100, Loss: 0.2233434942151819
Epoch 14/100, Loss: 0.22650208749941417
Epoch 15/100, Loss: 0.2358961317401666
Epoch 16/100, Loss: 0.22140807453747635
Epoch 17/100, Loss: 0.22250934628831162
Epoch 18/100, Loss: 0.20836820866871666
Epoch 19/100, Loss: 0.21283140144014095
Epoch 20/100, Loss: 0.22537827066012792
Epoch 21/100, Loss: 0.18720937012643604
Epoch 22/100, Loss: 0.20282919230041924
Epoch 23/100, Loss: 0.218539413023781
Epoch 24/100, Loss: 0.21176832202044163
Epoch 25/100, Loss: 0.24238447177704875
Epoch 26/100, Loss: 

# **Evaluation**

In [11]:
def evaluate(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().tolist())
    return predictions

predictions = evaluate(model, test_loader, device=device)
index_to_class = {0: 'wheat', 1: 'oats', 2: 'flax', 3: 'barley'}

# Конвертировать в названия
class_predictions = [index_to_class[pred] for pred in predictions]

test_df = pd.read_csv(test_csv_path)
test_df['class'] = class_predictions
test_df.to_csv('/content/drive/MyDrive/NeuralNetworks/predictions5.csv', index=False)